## Daily Reports Script

In [ ]:
import pandas as pd

## Import Files

In [ ]:
# Insert Date that match report file names
report_date = '10.4'

# File paths set for location of files
amt_file_path = (r'#')
inventory_file_path = (r'# {} #').format(report_date) # .format notation for varied dates
sales_forecast_path = (r'#')
units_path = (r'# {} #').format(report_date) # .format notation for varied dates

# Variable assignment and import for files
amt = pd.read_excel(amt_file_path)
inventory = pd.read_excel(inventory_file_path)
sales_forecast = pd.read_excel(sales_forecast_path)
units = pd.read_excel(kgp_pos_units_path)

## Customizing Data

In [ ]:
# Selecting Fiscal Weeks for Sum Total
sales_forecast_weeks = list(sales_forecast)
del sales_forecast_weeks[0:8] # The 8 will need to increase weekly to remove old fiscal weeks from sum

# New column for sum total of fiscal week selections 
sales_forecast['Forecast'] = sales_forecast[sales_forecast_weeks].sum(axis=1)
sales_forecast.rename(columns={'STR_NBR': 'STR NBR',
                              'SKU_NBR': 'Sku'}, inplace=True)

# New sales_forecast table with only necessary columns
columnslist = ['STR NBR', 'Sku', 'THD Forecast']
sales_forecast = sales_forecast[columnslist]

# Name changes to match amt file column names
inventory.rename(columns={'Store Nbr': 'STR NBR',
                         'SKU Nbr': 'Sku',
                         'Str OH Units Dly': 'OH'}, inplace = True)

# Removing unnecessary columns
inventory.drop(columns = ['SKU Name', 'Str OO Units Dly'], inplace = True) 

# Name change to match amt file column names
units.rename(columns={'Store': 'STR NBR',
                              'Item': 'Sku',
                              2018: 'Units'}, inplace=True)

## Merge Data Files

In [ ]:
# Merging OH data to AMT
combo = amt.merge(inventory,
                  how='left',
                  on=['STR NBR', 'Sku'])

# Second merge, deleting unnecessary columns, and renaming to match AMT
combo_b = combo.merge(units,
                      how='left',
                      left_on=['STR NBR', 'Old Sku'],
                      right_on=['STR NBR', 'Sku'])
del combo_b['Sku_y']
combo_b.rename(columns={'Sku_x': 'Sku'}, inplace=True)

# Merging THD_Forecast to new OH and AMT
daily_reports = combo_b.merge(sales_forecast,
                              how='left',
                              on=['STR NBR', 'Sku'])
daily_reports = daily_reports.fillna(0)

## Export Report

In [ ]:
export = pd.ExcelWriter(r'# '+ report_date +' Daily Reports.xlsx',
                        engine = 'xlsxwriter')

daily_reports.to_excel(export, index=False)

export.save()